<h1>3장 Pandas로 데이터 가공하기</h1>

<h2>계층적 인덱싱</h2>

In [61]:
import pandas as pd
import numpy as np

<b>다중 인덱스된 Series</b>

<b>나쁜 방식</b>

In [10]:
index = [('California', 2000),('California', 2010),
         ('New York', 2000),('New York', 2010),
         ('Texas', 2000),('Texas', 2010)]
populations = [33871648, 37253956,
              18976457, 19378102,
              20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [11]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [13]:
pop[(i for i in pop.index if i[1] == 2010)]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

<b>더 나은 방식:Pandas Multindex</b>

In [17]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [19]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [20]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

<b>Multiindex: 추가 차원</b>

In [21]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [22]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [23]:
pop_df = pd.DataFrame({'total': pop,
                      'under 18': [926789, 928494,
                                  4687342, 431033,
                                  590631, 6879014]})
pop_df

total  under 18
California 2000  33871648    926789
           2010  37253956    928494
New York   2000  18976457   4687342
           2010  19378102    431033
Texas      2000  20851820    590631
           2010  25145561   6879014

<b>MultiIndex 생성 메서드</b>

In [28]:
df = pd.DataFrame(np.random.rand(4,2), index=[['a','a','b','b'], [1,2,1,2]], columns=['data1', 'data2'])
df

data1     data2
a 1  0.367286  0.752681
  2  0.187242  0.011916
b 1  0.150645  0.268909
  2  0.511161  0.893491

In [31]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102
       }
data

{('California', 2000): 33871648,
 ('California', 2010): 37253956,
 ('New York', 2000): 18976457,
 ('New York', 2010): 19378102,
 ('Texas', 2000): 20851820,
 ('Texas', 2010): 25145561}

In [32]:
pd.Series(data)

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

<b>명시적 MultiIndex 생성자</b>

In [33]:
pd.MultiIndex.from_arrays([['a','a','b','b'], [1,2,1,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [34]:
pd.MultiIndex.from_tuples([('a',1), ('a',2), ('b',1), ('b',2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [40]:
pd.MultiIndex.from_product([['a','b'],[1,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [44]:
pd.MultiIndex(levels=[['a','b'],[1,2]],
              labels=[[0,0,1,1],[0,1,0,1]]
             )

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

<b>MultiIndex 레벨 이름</b>

In [108]:
pop.index.names = ['State', 'year']
pop

State       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

<b>열의 MultiIndex</b>

In [109]:
# 계층적 인덱스와 열
index = pd.MultiIndex.from_product([[2013, 2014], [1,2]],
                                  names=['year', 'visit'])
index

MultiIndex(levels=[[2013, 2014], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['year', 'visit'])

In [110]:
columns = pd.MultiIndex.from_product([['Bob','Guido','Sue'],['HR', 'Temp']],
                                    names=['subject', 'type'])
columns

MultiIndex(levels=[['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]],
           names=['subject', 'type'])

In [111]:
# 일부 데이터 모형 만들기 
data = np.round(np.random.randn(4,6),1)
data

array([[ 0.4,  0.1,  0.3,  0.1, -0.5,  2.3],
       [ 0.3,  2. , -1. ,  1.6, -0.1, -0.8],
       [ 1. ,  0.3, -1.9,  0.2,  0. , -0.3],
       [-0.8,  0.9,  0. , -1.3, -2.4, -0. ]])

In [112]:
data[:, ::2] *= 10
data

array([[  4. ,   0.1,   3. ,   0.1,  -5. ,   2.3],
       [  3. ,   2. , -10. ,   1.6,  -1. ,  -0.8],
       [ 10. ,   0.3, -19. ,   0.2,   0. ,  -0.3],
       [ -8. ,   0.9,   0. ,  -1.3, -24. ,  -0. ]])

In [113]:
data += 37
data

array([[41. , 37.1, 40. , 37.1, 32. , 39.3],
       [40. , 39. , 27. , 38.6, 36. , 36.2],
       [47. , 37.3, 18. , 37.2, 37. , 36.7],
       [29. , 37.9, 37. , 35.7, 13. , 37. ]])

In [114]:
# DataFram 생성하기
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      41.0  37.1  40.0  37.1  32.0  39.3
     2      40.0  39.0  27.0  38.6  36.0  36.2
2014 1      47.0  37.3  18.0  37.2  37.0  36.7
     2      29.0  37.9  37.0  35.7  13.0  37.0

In [115]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      40.0  37.1
     2      27.0  38.6
2014 1      18.0  37.2
     2      37.0  35.7

<b>MultiIndex 인덱싱 및 슬라이딩</b>

<b>다중 인덱스를 가진 Series</b>

In [116]:
pop

State       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [117]:
pop['California', 2000]

33871648

In [118]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [119]:
pop.loc['California':'New York']

State       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [120]:
pop[:, 2000]

State
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [121]:
pop[pop> 2200000]

State       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [122]:
pop[['California', 'Texas']] # 팬시 인덱싱(Dancy Indexing)

State       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

<b>다중 인덱스를 가진 DataFrame</b>

In [123]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      41.0  37.1  40.0  37.1  32.0  39.3
     2      40.0  39.0  27.0  38.6  36.0  36.2
2014 1      47.0  37.3  18.0  37.2  37.0  36.7
     2      29.0  37.9  37.0  35.7  13.0  37.0

In [124]:
health_data['Guido', 'HR']

year  visit
2013  1        40.0
      2        27.0
2014  1        18.0
      2        37.0
Name: (Guido, HR), dtype: float64

In [125]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      41.0  37.1
     2      40.0  39.0

In [126]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        41.0
      2        40.0
2014  1        47.0
      2        29.0
Name: (Bob, HR), dtype: float64

In [127]:
health_data.loc[(:, 1), (:, 'HR')] # 튜플 내에 슬라이스를 생성하면 구문 에러가 발생한다.

SyntaxError: invalid syntax (<ipython-input-127-97e94ddc3690>, line 1)

In [128]:
idx = pd.IndexSlice #slice()를 사용해 슬라이스를 명시적으로 만들면 가능하고, IndexSlice 객체를 사용하는 것도 된다.
health_data.loc[idx[:,1], idx[:,'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,41.0,40.0,32.0
2014,1,47.0,18.0,37.0


<b>다중 인덱스 재정렬하기</b>

<b>정렬된 인덱스와  정렬되지 않은 인덱스</b>

In [129]:
index = pd.MultiIndex.from_product([['a','c','b'],[1,2]])
index

MultiIndex(levels=[['a', 'b', 'c'], [1, 2]],
           labels=[[0, 0, 2, 2, 1, 1], [0, 1, 0, 1, 0, 1]])

In [130]:
data = pd.Series(np.random.rand(6), index = index )
data

a  1    0.393165
   2    0.487514
c  1    0.314646
   2    0.722313
b  1    0.243360
   2    0.586580
dtype: float64

In [131]:
data.index.names = ['char', 'int']
data

char  int
a     1      0.393165
      2      0.487514
c     1      0.314646
      2      0.722313
b     1      0.243360
      2      0.586580
dtype: float64

In [132]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)
    

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [133]:
data = data.sort_index()
data

char  int
a     1      0.393165
      2      0.487514
b     1      0.243360
      2      0.586580
c     1      0.314646
      2      0.722313
dtype: float64

In [134]:
data['a':'b']

char  int
a     1      0.393165
      2      0.487514
b     1      0.243360
      2      0.586580
dtype: float64

<b>인덱스 스태킹 및 언스태킹</b>

In [135]:
pop.unstack(level=0)

State,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [136]:
pop.unstack(level=1)

year,2000,2010
State,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [137]:
pop.unstack().stack()

State       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

<b>인덱스 설정 및 재설정</b>

In [138]:
pop_flat = pop.reset_index(name='populations')
pop_flat

,State,year,populations
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [139]:
pop_flat.set_index(['State', 'year'])

populations
State      year             
California 2000     33871648
           2010     37253956
New York   2000     18976457
           2010     19378102
Texas      2000     20851820
           2010     25145561

<b>다중 인덱스에서 데이터 집계</b>

In [141]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      41.0  37.1  40.0  37.1  32.0  39.3
     2      40.0  39.0  27.0  38.6  36.0  36.2
2014 1      47.0  37.3  18.0  37.2  37.0  36.7
     2      29.0  37.9  37.0  35.7  13.0  37.0

In [142]:
data_mean = health_data.mean(level='year') #중간값
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     40.5  38.05  33.5  37.85  34.0  37.75
2014     38.0  37.60  27.5  36.45  25.0  36.85

In [144]:
health_data.mean(axis = 1, level='type') 

type               HR       Temp
year visit                      
2013 1      37.666667  37.833333
     2      34.333333  37.933333
2014 1      34.000000  37.066667
     2      26.333333  36.866667